# Fetch wiki corpus
- [NLP 學習筆記](https://hackmd.io/Gw4TgIwEwhaAGYAzOAWKBWYtIYMyxbICGeUATMAKbFRA)
- [以 gensim 訓練中文詞向量](http://zake7749.github.io/2016/08/28/word2vec-with-gensim/)
- [tfidf 關鍵字擷取](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer)

In [1]:
import jieba
import time
import re

### Extract datas from wiki
- [維基百科:資料庫下載](https://zh.wikipedia.org/wiki/Wikipedia:%E6%95%B0%E6%8D%AE%E5%BA%93%E4%B8%8B%E8%BD%BD)
    - 文章含有部分 style tag 沒被移除
    - 含有不相關的字詞在文章中
    - 中英繁簡夾雜
    - 看起來不像一般的文章 (有些斷句的地方怪怪的)
- 下 `python3 WikiExtractor.py` 將 `datas/raw/zhwiki-20170801-pages-articles.xml.bz2` 轉成 `datas/wiki-texts.txt`
- 下 `opencc -i datas/wiki-texts.txt -o datas/wiki-texts-zht.txt` 將簡體全部轉繁
    - _hanziconv_ 轉的超爛 (e.g. 數學家 => 數學傢 / 一方面 => 一方麵)

### Tokenize
- [如何使用 JIEBA 結巴中文分詞程式](http://blog.fukuball.com/ru-he-shi-yong-jieba-jie-ba-zhong-wen-fen-ci-cheng-shi/)
- [中文 stop words](https://github.com/stopwords-iso/stopwords-zh)
- [jieba-zh_TW](https://github.com/ldkrsi/jieba-zh_TW.git)
- 之後用 word2vec 做 word embedding 是 fixed window size，故儘量用最少的字數達到意思
- stop words 詞庫 (有些奇怪不應該的字被加到停用字，要人工移除 e.g. 社会主义)
    - 留意，Token 中的這些字將被移除
    - **TODO** 更新 + 人工檢查 stop words 詞庫
    - https://github.com/stopwords-iso/stopwords-zh/blob/master/raw/baidu.txt
    - https://github.com/stopwords-iso/stopwords-zh/tree/master/raw
    - https://github.com/fxsjy/jieba/tree/master/extra_dict
    - https://gist.github.com/dreampuf/5548203
- 中文字典詞庫
    - ✓ https://github.com/fxsjy/jieba/tree/master/extra_dict
    - ✓ http://resources.publicense.moe.edu.tw/
    - https://raw.githubusercontent.com/NLPchina/ansj_seg/master/src/main/resources/bigramdict.dic
    - https://g0v.hackpad.tw/3du.tw-ZNwaun62BP4

In [22]:
stopwords_set = set()
with open('datas/dict/stop-words.txt', 'r') as f:
    stopwords_set.update([line.strip() for line in f])
stopwords_set.update(['\n', '\t', '\r'])

In [23]:
stopwords_set

{'\t',
 '\n',
 '\r',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '他',
 '他們',
 '你',
 '你們',
 '她',
 '她們',
 '妳',
 '妳們',
 '它',
 '它們',
 '您',
 '您們',
 '我',
 '我們',
 '︿',
 '！',
 '＃',
 '＄',
 '％',
 '＆',
 '（',
 '）',
 '＊',
 '＋',
 '，',
 '０',
 '１',
 '２',
 '３',
 '４',
 '５',
 '６',
 '７',
 '８',
 '９',
 '：',
 '；',
 '＜',
 '＞',
 '？',
 '＠',
 '［',
 '］',
 '｛',
 '｜',
 '｝',
 '～',
 '￥'}

In [4]:
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')

Building prefix dict from /home/sunset/word_contest/datas/dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u849ecfdca27003d306f39ca004b82b5b.cache
Loading model cost 1.180 seconds.
Prefix dict has been built succesfully.


In [24]:
# Show one "sentence" for preview
with open('datas/wiki-texts-zht.txt', 'r') as fi:
    for line in fi:
        line = line
        print(line)
        print([word for word in jieba.cut(line) if word not in stopwords_set])
        break

歐幾裏得 西元前三世紀的希臘數學家 現在被認為是幾何之父 此畫為拉斐爾的作品 雅典學院 數學 是利用符號語言研究數量 結構 變化以及空間等概唸的一門學科 從某種角度看屬於形式科學的一種 數學透過抽象化和邏輯推理的使用 由計數 計算 數學家們拓展這些概念 對數學基本概唸的完善 早在古埃及 而在古希臘那裡有更為嚴謹的處理 從那時開始 數學的發展便持續不斷地小幅進展 世紀的文藝復興時期 致使數學的加速發展 直至今日 今日 數學使用在不同的領域中 包括科學 工程 醫學和經濟學等 有時亦會激起新的數學發現 並導致全新學科的發展 數學家也研究純數學 就是數學本身的實質性內容 而不以任何實際應用為目標 雖然許多研究以純數學開始 但其過程中也發現許多應用之處 詞源 西方語言中 數學 一詞源自於古希臘語的 其有 學習 學問 科學 數學研究 即使在其語源內 其形容詞 意思為 和學習有關的 用功的 亦會被用來指 數學的 其在英語中表面上的複數形式 及在法語中的表面複數形式 可溯至拉丁文的中性複數 由西塞羅譯自希臘文複數 此一希臘語被亞里士多德拿來指 萬物皆數 的概念 漢字表示的 數學 一詞大約產生於中國宋元時期 多指象數之學 但有時也含有今天上的數學意義 例如 秦九韶的 數學九章 永樂大典 數書九章 也被宋代周密所著的 癸辛雜識 記爲 數學大略 數學通軌 明代柯尚遷著 數學鑰 清代杜知耕著 數學拾遺 清代丁取忠撰 直到 經過中國數學名詞審查委員會研究 算學 數學 兩詞的使用狀況後 確認以 數學 表示今天意義上的數學含義 歷史 奇普 印加帝國時所使用的計數工具 瑪雅數字 數學有着久遠的歷史 中國古代的六藝之一就有 數學一詞在西方有希臘語詞源 mathematikós 意思是 學問的基礎 源於 máthema 科學 知識 學問 時間的長短等抽象的數量關係 比如時間單位有日 季節和年等 算術 加減乘除 也自然而然地產生了 歷史上曾有過許多不同的記數系統 在最初有歷史記錄的時候 為瞭解數字間的關係 為了測量土地 以及為了預測天文事件而形成的 結構 空間及時間方面的研究 到了 世紀 算術 微積分的概念也在此時形成 隨着數學轉向形式化 從古至今 數學便一直不斷地延展 且與科學有豐富的相互作用 兩者的發展都受惠於彼此 在歷史上有著許多數學發現 並且直至今日都不斷地有新的發現 據mikhail sevr

In [25]:
with open('datas/wiki-texts-zht.txt', 'r') as fi, open('datas/wiki-seg.txt', 'w') as fo:
    start_time = time.time()
    for i, line in enumerate(fi):
        fo.write(' '.join([word for word in jieba.cut(line) if word not in stopwords_set]) + '\n')
        if i % 1000 == 0:
            print('Finished %3dk lines / elapsed time %10.2f' % (i/1000, time.time() - start_time), end='\r')